Автор: Найденов Алексей

# Задача
Построить модель, предсказывающую категорию объявления по его заголовку, описанию и цене. Метрика для оценки качества - accuracy. Результат скоринга файла test.csv с помощью предложенного классификатора (csv-файл с двумя столбцами: item_id, category_id).
Категории имеют иерархическую структуру, описанную в файле сategory.csv. Необходимо посчитать также accuracy модели на каждом уровне иерархии.

# Описание
Лучшее найденное решение довольно простое:

## 1. Предобработка данных

Поля `title` и `description`: удалить все знаки препинания, оставшиеся слова привести к нормальной форме (библиотека `pymorphy2`).

## 2. Токенизация текста (+ TF-IDF)

Использовать токены только из поля `title` (обучить на `title` и "натравить" на `title` + `description`). Неочевидное решение выбрано на основе того, что с большой долей вероятности значимые слова встречаются в заголовке, а описание помимо прочего содержит много "мусора".

## 3. Классификация

Лучше всего показал себя метод опорных векторов с линейным ядром (`LinerSVC`). Гиперпараметры подбирались в `GridSearch`.

In [1]:
# обработка текста
import re
import pymorphy2
# работа с данными
import pandas as pd
import numpy as np
# анализ данных
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

# Данные
Все данные хранятся в директории `data`, смежной с текущей (т.е. путь "../data/").

Обучающий набор ("train.csv"):
- `item_id` - ID объявления
- `title` - заголовок объявления
- `description` - описание объявления
- `price` - цена
- `category_id` - категория (зависимая переменная)

Дополнительный набор, описывающий иерархию категорий:
- `category_id` - ID категории
- `name` - название категории, где через символ "|" перечислена вся иерархия категории (например, "Бытовая электроника|Телефоны|Samsung")


## Замечание
В случае если параметр `HAS_CLEAN` установлен в `True`, то мы пропускаем длительный этап очистки данных, и загружаем готовый файл непосредственно из директории `data`.

In [2]:
# HAS_CLEAN = True
HAS_CLEAN = False

In [3]:
"""CONFIG"""
# директория с данными
PATH_DATA = "../data/"
# обучаемый набор
PATH_TRAIN = PATH_DATA + ("train.csv" if not HAS_CLEAN else "train_normal.csv")
# тестовый набор
PATH_TEST = PATH_DATA + ("test.csv" if not HAS_CLEAN else "test_normal.csv")
# категории
PATH_CATS = PATH_DATA + "category.csv"
# выходное поле
target_label = u'category_id'
# предикторы
pred_labels = [u'title', u'description']
title_label = u'title'

In [4]:
"""Чтение данных"""
dtrain = pd.read_csv(PATH_TRAIN)
dtest = pd.read_csv(PATH_TEST)
cats = pd.read_csv(PATH_CATS) 
target = dtrain[target_label]
title = dtrain[title_label]

# Функции

In [5]:
"""Functions"""
from sklearn.metrics import accuracy_score

# для построенной на (x, y) модели возвращает accuracy на (x_test, y_test)
def check_accuracy(clf, x, y, x_test, y_test):
    clf.fit(x, y)
    t = clf.predict(x_test)
    return accuracy_score(y_test, t)

# возвращает позицию n-го найденного символа char в строке line  
def find_nth(line, char, n):
    start = line.find(char)
    while start >= 0 and n > 1:
        start = line.find(char, start+len(char))
        n -= 1
    return start

# возвращает обрезанное имя категории, т.е.категорию уровня n 
def get_hierarchical_cat(cat="", n=1):
    line = cat.decode("utf-8")
    index = find_nth(line, "|", n)
    return line[:index] if index > 0 else line

# Предобработка исходных данных

In [6]:
"""CLEAN DATA"""
pattern_not_word = re.compile(u'[^a-zA-Zа-яёЁА-Я0-9_]+')
morph = pymorphy2.MorphAnalyzer()
"""Удаление знаков препинания"""
def filter_punc_mark(line):   
    tokens = pattern_not_word.sub(' ', line).strip()
    return tokens
"""Получение нормальной формы слова (по-умолчанию первая форма слова)"""
def normal_form(line):
    return ' '.join([morph.parse(w)[0].normal_form for w in line.split()]) 

def get_concat_df(df, cols):
    res = np.str("")
    for col in cols:
        res += df[col] + np.str(" ")
    return res

Наборы данных в кодировке UTF-8 без BOM. Для обработки текста приходится декодировать.

In [7]:
"""Очитска трейна"""
if not HAS_CLEAN:
    for col in pred_labels:
        dtrain[col] = dtrain[col].str.decode('utf-8').apply(filter_punc_mark).apply(normal_form).str.encode('utf-8')

for col in pred_labels:
    dtrain[col].fillna('', inplace=True)

In [8]:
"""Очитска теста"""
if not HAS_CLEAN:
    for col in pred_labels:
        dtest[col] = dtest[col].str.decode('utf-8').apply(filter_punc_mark).apply(normal_form).str.encode('utf-8')

for col in pred_labels:
    dtest[col].fillna('', inplace=True)

# Токенизация текста (+ TF-IDF)
`TfidfVectorizer` - векторизирует и транфомирует текст

In [9]:
# Определили векторизатор
vect = TfidfVectorizer(
    analyzer='word', 
    lowercase=True,
    ngram_range=(1,2),
    token_pattern=u'(?u)\\b\\w+\\b',
    binary='True',
    min_df=1
)
# Обучили на `title`
vect.fit(title)
# Преобразовали конкатенированные предикторы на трейне (`title` и `description`)
X_train = vect.transform(get_concat_df(dtrain, pred_labels))
# Преобразовали конкатенированные предикторы на тесте (`title` и `description`)
X_test = vect.transform(get_concat_df(dtest, pred_labels))

# Кроссвалидация

Строим кроссвалидацию модели на всех уровнях иерархической структуры. Каждый уровень иерархии находим просто: обрезаем название категории по нужному по счету символу "|" (функция `get_hierarchical_cat`)

In [10]:
"""CrossValidation"""
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score

# Выполнение cv на каждом уровне иерархической структуры 
# (по-умолчанию для полной иерархии)
def compute_hierarchical_cv(hierarchical_level=0):
    if hierarchical_level > 0:
        xtarget = pd.merge(
            pd.DataFrame({'category_id': target}), 
            right=cats, 
            how='left', 
            on='category_id'
        )['name'].apply(lambda x: get_hierarchical_cat(x, n=hierarchical_level))
        print "Hierarchy level: %d" % hierarchical_level
    else:
        xtarget = target
        print "Full hierarchy"
    compute_cv(xtarget)
    
# Ввели свою cv так как специфично подготавливаем набор данных, 
# а свой Classifier писать не быстрее
def compute_cv(target): 
    skf = StratifiedKFold(target, n_folds=3, random_state=42)
    scores=[]
    for train_index, test_index in skf:
        # Определили векторизатор
        vect = TfidfVectorizer(
            analyzer='word', 
            lowercase=True,
            ngram_range=(1,2),
            token_pattern=u'(?u)\\b\\w+\\b',
            binary='True',
            min_df=1
        )
        # Обучили на `title`
        vect.fit(title[train_index])
        # Преобразовали конкатенированные предикторы на трейне (`title` и `description`)
        X_train = vect.transform(get_concat_df(dtrain, pred_labels)[train_index])
        # Преобразовали конкатенированные предикторы на тесте (`title` и `description`)
        X_test = vect.transform(get_concat_df(dtrain, pred_labels)[test_index])
        y_train, y_test = target[train_index], target[test_index]

        clf = LinearSVC(
            C=0.5,
            tol=1e-05,
            max_iter=100,
            random_state=42
        )

        scores.append(check_accuracy(clf, X_train, y_train, X_test, y_test))
    print "accuracy_score: %f" % np.mean(scores)

Глубина иерархической структуры = 5.

In [11]:
for i in range(5):
    compute_hierarchical_cv(i)

Full hierarchy
accuracy_score: 0.888151
Hierarchy level: 1
accuracy_score: 0.960524
Hierarchy level: 2
accuracy_score: 0.945350
Hierarchy level: 3
accuracy_score: 0.891857
Hierarchy level: 4
accuracy_score: 0.888151


# Предсказание категории на тестовом множестве

In [12]:
clf = LinearSVC(
    C=0.5,
    tol=1e-05,
    max_iter=100
)
# Обучили на трейне
clf.fit(X_train, target)
# Предсказали для теста и записали в файл
preds = clf.predict(X_test)

In [13]:
pd.DataFrame({
        'item_id': dtest['item_id'],
        'category_id': preds
        }, columns=['item_id', 'category_id']).to_csv("Submit.csv", index=False)

# Возможные направления для улучшения модели

1. Изменить паттерн слова (`u'[^a-zA-Zа-яёЁА-Я0-9_]+'`) в функции удаления знаков препинания. Например, не убирать дефизы.
2. Использовать цену в модели классификатора.
3. Использовать стекинг:

 - Обучить новую модель на ответах-метках классификаторов первого уровня.
 - Обучить новую модель (XGBoost) на непрерывных коэффициентах для каждого класса при нескольких классификаторах. Итого будет 54\*n непрерывных полей, где n - число классификаторов. 